<a href="https://colab.research.google.com/github/darshlukkad/AutoGluon/blob/main/Ieee_cis_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U pip setuptools wheel
!pip install -U autogluon autogluon.timeseries

  Using cached autogluon-1.4.0-py3-none-any.whl.metadata (11 kB)
  Using cached autogluon.timeseries-1.4.0-py3-none-any.whl.metadata (12 kB)
  Using cached autogluon.core-1.4.0-py3-none-any.whl.metadata (12 kB)
  Using cached autogluon.features-1.4.0-py3-none-any.whl.metadata (11 kB)
  Using cached autogluon.tabular-1.4.0-py3-none-any.whl.metadata (16 kB)
  Using cached autogluon.multimodal-1.4.0-py3-none-any.whl.metadata (13 kB)
  Using cached torch-2.7.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached lightning-2.5.5-py3-none-any.whl.metadata (39 kB)
  Using cached pytorch_lightning-2.5.5-py3-none-any.whl.metadata (20 kB)
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached gluonts-0.16.2-py3-none-any.whl.metadata (9.8 kB)
  Using cached statsforecast-2.0.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (29 kB)
  Using cached mlforecast-0.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached utilsforecast-0.2.1

In [1]:
# Install gdown and create the target directory
!pip -q install gdown
!mkdir -p /content/data

# Download the entire shared folder to /content/data
!gdown --folder --fuzzy "https://drive.google.com/drive/folders/1K6Zdl_rt8AH0XRE4ww_jbvUtP3Zm91Xa?usp=sharing" -O /content/data

# Quick check
!ls -lah /content/data


Retrieving folder contents
Processing file 1UrYAq4wz4udNEBBVxz3OfgLo_MR431c8 sample_submission.csv
Processing file 1q9umJKauLdYB3SJAgNKRosCTyInm_LS9 test_identity.csv
Processing file 1txlRx-kkHrCEMSWJvFtAIycuvab2WWz4 test_transaction.csv
Processing file 1Mb3600K-3LlU-R9cfxSIcLJCg2I58BPK train_identity.csv
Processing file 1Kcr6kwewhCEEVI9MutBIoRIfnCGPLYpq train_transaction.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1UrYAq4wz4udNEBBVxz3OfgLo_MR431c8
To: /content/data/sample_submission.csv
100% 6.08M/6.08M [00:00<00:00, 62.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1q9umJKauLdYB3SJAgNKRosCTyInm_LS9
To: /content/data/test_identity.csv
100% 25.8M/25.8M [00:00<00:00, 67.9MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1txlRx-kkHrCEMSWJvFtAIycuvab2WWz4
From (redirected): https://drive.google.com/uc?id=1txlRx-kkHrCEMSWJvFtAIycuvab2WWz4&confi

In [4]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = '/content/data/'  # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'AutoGluonModels/'  # where to store trained models

train_identity = pd.read_csv(directory+'train_identity.csv')
train_transaction = pd.read_csv(directory+'train_transaction.csv')

In [5]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

# ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
# Take a 20% stratified sample to speed up training  # ← CHANGED
train_data_sample = (
    train_data.groupby(label, group_keys=False)
              .apply(lambda x: x.sample(frac=0.2, random_state=42))  # ← CHANGED
)
print(f"Full train rows: {len(train_data):,} | Sampled rows: {len(train_data_sample):,}")  # ← CHANGED
# ↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑

Full train rows: 590,540 | Sampled rows: 118,108


/tmp/ipython-input-3243189372.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(frac=0.2, random_state=42))  # ← CHANGED


In [6]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data_sample,  # ← CHANGED (was: train_data)
    presets='medium_quality',
    time_limit=3600
)

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          8
GPU Count:          1
Memory Avail:       43.61 GB / 50.99 GB (85.5%)
Disk Space Avail:   188.31 GB / 235.68 GB (79.9%)
Presets specified: ['medium_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': False}
Full kwargs:
{'_experimental_dynamic_hyperparameters': False,
 '_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': False,
 'calibrate': 'auto',
 'delay_bag_sets': False,
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_callbacks': False,
             'enable_ray_logging': True,
             'holdout_data': None,
             'holdout_frac': 0.1

[50]	valid_set's binary_logloss: 0.106058
[100]	valid_set's binary_logloss: 0.098899
[150]	valid_set's binary_logloss: 0.0966721
[200]	valid_set's binary_logloss: 0.0940842
[250]	valid_set's binary_logloss: 0.0926591
[300]	valid_set's binary_logloss: 0.0915665
[350]	valid_set's binary_logloss: 0.0907962
[400]	valid_set's binary_logloss: 0.0908043
[450]	valid_set's binary_logloss: 0.090198
[500]	valid_set's binary_logloss: 0.0900936
[550]	valid_set's binary_logloss: 0.0901112
[600]	valid_set's binary_logloss: 0.0897297
[650]	valid_set's binary_logloss: 0.0896885
[700]	valid_set's binary_logloss: 0.0894876
[750]	valid_set's binary_logloss: 0.0889608
[800]	valid_set's binary_logloss: 0.088886
[850]	valid_set's binary_logloss: 0.0888104
[900]	valid_set's binary_logloss: 0.0886612
[950]	valid_set's binary_logloss: 0.0885508
[1000]	valid_set's binary_logloss: 0.0885021
[1050]	valid_set's binary_logloss: 0.0888248
[1100]	valid_set's binary_logloss: 0.0889373
[1150]	valid_set's binary_logloss:

Saving /content/data/AutoGluonModels/models/LightGBMXT/model.pkl
Saving /content/data/AutoGluonModels/utils/attr/LightGBMXT/y_pred_proba_val.pkl
	0.8868	 = Validation score   (roc_auc)
	30.71s	 = Training   runtime
	0.14s	 = Validation runtime
	18014.1	 = Inference  throughput (rows/s | 2500 batch size)
Saving /content/data/AutoGluonModels/models/trainer.pkl
Fitting model: LightGBM ... Training model for up to 3560.41s of the 3560.40s of remaining time.
	Fitting LightGBM with 'num_gpus': 0, 'num_cpus': 4
	Fitting with cpus=4, gpus=0, mem=2.0/42.8 GB
	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05}


[50]	valid_set's binary_logloss: 0.102547
[100]	valid_set's binary_logloss: 0.0964554
[150]	valid_set's binary_logloss: 0.0930342
[200]	valid_set's binary_logloss: 0.0912133
[250]	valid_set's binary_logloss: 0.0906228
[300]	valid_set's binary_logloss: 0.0905111
[350]	valid_set's binary_logloss: 0.0898684
[400]	valid_set's binary_logloss: 0.0892858
[450]	valid_set's binary_logloss: 0.0889965
[500]	valid_set's binary_logloss: 0.089125
[550]	valid_set's binary_logloss: 0.0892667
[600]	valid_set's binary_logloss: 0.0893957


Saving /content/data/AutoGluonModels/models/LightGBM/model.pkl
Saving /content/data/AutoGluonModels/utils/attr/LightGBM/y_pred_proba_val.pkl
	0.887	 = Validation score   (roc_auc)
	16.79s	 = Training   runtime
	0.05s	 = Validation runtime
	46981.7	 = Inference  throughput (rows/s | 2500 batch size)
Saving /content/data/AutoGluonModels/models/trainer.pkl
Fitting model: RandomForestGini ... Training model for up to 3543.54s of the 3543.53s of remaining time.
	Fitting RandomForestGini with 'num_gpus': 0, 'num_cpus': 8
	Fitting with cpus=8, gpus=0, mem=0.1/42.7 GB
Saving /content/data/AutoGluonModels/models/RandomForestGini/model.pkl
Saving /content/data/AutoGluonModels/utils/attr/RandomForestGini/y_pred_proba_val.pkl
	0.8748	 = Validation score   (roc_auc)
	42.04s	 = Training   runtime
	0.12s	 = Validation runtime
	20500.2	 = Inference  throughput (rows/s | 2500 batch size)
Saving /content/data/AutoGluonModels/models/trainer.pkl
Fitting model: RandomForestEntr ... Training model for up to

...

Saving /content/data/AutoGluonModels/models/CatBoost/model.pkl
Saving /content/data/AutoGluonModels/utils/attr/CatBoost/y_pred_proba_val.pkl
	0.8699	 = Validation score   (roc_auc)
	765.12s	 = Training   runtime
	0.04s	 = Validation runtime
	60950.6	 = Inference  throughput (rows/s | 2500 batch size)
Saving /content/data/AutoGluonModels/models/trainer.pkl
Fitting model: ExtraTreesGini ... Training model for up to 2703.41s of the 2703.40s of remaining time.
	Fitting ExtraTreesGini with 'num_gpus': 0, 'num_cpus': 8
	Fitting with cpus=8, gpus=0, mem=0.1/42.1 GB
Saving /content/data/AutoGluonModels/models/ExtraTreesGini/model.pkl
Saving /content/data/AutoGluonModels/utils/attr/ExtraTreesGini/y_pred_proba_val.pkl
	0.8659	 = Validation score   (roc_auc)
	35.88s	 = Training   runtime
	0.12s	 = Validation runtime
	20549.8	 = Inference  throughput (rows/s | 2500 batch size)
Saving /content/data/AutoGluonModels/models/trainer.pkl
Fitting model: ExtraTreesEntr ... Training model for up to 2667.14

[0]	validation_0-logloss:0.13443
[50]	validation_0-logloss:0.09805
[100]	validation_0-logloss:0.09580
[150]	validation_0-logloss:0.09456
[200]	validation_0-logloss:0.09332
[250]	validation_0-logloss:0.09311
[300]	validation_0-logloss:0.09301
[350]	validation_0-logloss:0.09307
[400]	validation_0-logloss:0.09293
[450]	validation_0-logloss:0.09267
[500]	validation_0-logloss:0.09239
[550]	validation_0-logloss:0.09245
[600]	validation_0-logloss:0.09242
[650]	validation_0-logloss:0.09294
[700]	validation_0-logloss:0.09265
[701]	validation_0-logloss:0.09268


Saving /content/data/AutoGluonModels/models/XGBoost/model.pkl
Saving /content/data/AutoGluonModels/utils/attr/XGBoost/y_pred_proba_val.pkl
	0.8728	 = Validation score   (roc_auc)
	40.53s	 = Training   runtime
	0.08s	 = Validation runtime
	31198.7	 = Inference  throughput (rows/s | 2500 batch size)
Saving /content/data/AutoGluonModels/models/trainer.pkl
Fitting model: NeuralNetTorch ... Training model for up to 2404.26s of the 2404.25s of remaining time.
	Fitting NeuralNetTorch with 'num_gpus': 0, 'num_cpus': 4
	Fitting with cpus=4, gpus=0, mem=1.7/41.5 GB
Tabular Neural Network treats features as the following types:
{
    "continuous": [
        "TransactionID",
        "TransactionDT",
        "card1",
        "card2",
        "addr1",
        "D9",
        "D15",
        "V4",
        "V5",
        "V10",
        "V11",
        "V12",
        "V13",
        "V19",
        "V20",
        "V29",
        "V35",
        "V36",
        "V48",
        "V53",
        "V54",
        "V61",


[50]	valid_set's binary_logloss: 0.103691
[100]	valid_set's binary_logloss: 0.0956807
[150]	valid_set's binary_logloss: 0.0929781
[200]	valid_set's binary_logloss: 0.0915392
[250]	valid_set's binary_logloss: 0.0908786
[300]	valid_set's binary_logloss: 0.0910374
[350]	valid_set's binary_logloss: 0.0915381


Saving /content/data/AutoGluonModels/models/LightGBMLarge/model.pkl
Saving /content/data/AutoGluonModels/utils/attr/LightGBMLarge/y_pred_proba_val.pkl
	0.891	 = Validation score   (roc_auc)
	26.49s	 = Training   runtime
	0.05s	 = Validation runtime
	47082.8	 = Inference  throughput (rows/s | 2500 batch size)
Saving /content/data/AutoGluonModels/models/trainer.pkl
Loading: /content/data/AutoGluonModels/utils/attr/NeuralNetTorch/y_pred_proba_val.pkl
Loading: /content/data/AutoGluonModels/utils/attr/CatBoost/y_pred_proba_val.pkl
Loading: /content/data/AutoGluonModels/utils/attr/LightGBMLarge/y_pred_proba_val.pkl
Loading: /content/data/AutoGluonModels/utils/attr/LightGBMXT/y_pred_proba_val.pkl
Loading: /content/data/AutoGluonModels/utils/attr/ExtraTreesEntr/y_pred_proba_val.pkl
Loading: /content/data/AutoGluonModels/utils/attr/NeuralNetFastAI/y_pred_proba_val.pkl
Loading: /content/data/AutoGluonModels/utils/attr/LightGBM/y_pred_proba_val.pkl
Loading: /content/data/AutoGluonModels/utils/att

In [7]:
results = predictor.fit_summary()

Loading: /content/data/AutoGluonModels/models/LightGBMXT/model.pkl
Loading: /content/data/AutoGluonModels/models/LightGBM/model.pkl
Loading: /content/data/AutoGluonModels/models/RandomForestGini/model.pkl
Loading: /content/data/AutoGluonModels/models/RandomForestEntr/model.pkl
Loading: /content/data/AutoGluonModels/models/CatBoost/model.pkl
Loading: /content/data/AutoGluonModels/models/ExtraTreesGini/model.pkl
Loading: /content/data/AutoGluonModels/models/ExtraTreesEntr/model.pkl
Loading: /content/data/AutoGluonModels/models/NeuralNetFastAI/model.pkl
Loading: /content/data/AutoGluonModels/models/NeuralNetFastAI/model-internals.pkl
Loading: /content/data/AutoGluonModels/models/XGBoost/model.pkl
Loading: /content/data/AutoGluonModels/models/NeuralNetTorch/model.pkl
Loading: /content/data/AutoGluonModels/models/LightGBMLarge/model.pkl
Loading: /content/data/AutoGluonModels/models/WeightedEnsemble_L2/model.pkl


*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.908317     roc_auc       0.463439  305.823139                0.000825           0.134539            2       True         12
1         LightGBMLarge   0.891045     roc_auc       0.053098   26.494978                0.053098          26.494978            1       True         11
2              LightGBM   0.886968     roc_auc       0.053212   16.790235                0.053212          16.790235            1       True          2
3            LightGBMXT   0.886782     roc_auc       0.138780   30.708149                0.138780          30.708149            1       True          1
4      RandomForestEntr   0.880635     roc_auc       0.109567   32.361055                0.109567          32.361055            1       True          4
5       NeuralNetFastAI   

In [8]:
test_identity = pd.read_csv(directory+'test_identity.csv')
test_transaction = pd.read_csv(directory+'test_transaction.csv')
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')  # same join applied to training files

In [ ]:
y_predproba = predictor.predict_proba(test_data)
y_predproba.head(5)  # some example predicted fraud-probabilities

In [10]:
predictor.positive_class

1

In [ ]:
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)

In [ ]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)